# Day 2: Transaction Costs

## Learning Objectives
- Model realistic transaction costs
- Calculate portfolio turnover
- Understand impact on strategy performance

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("✅ Libraries loaded!")
print("📚 Day 2: Transaction Costs")

In [ ]:
# ============================================================
# GENERATE DATA & TRAIN MODEL
# ============================================================

print("PREPARING DATA")
print("="*60)

np.random.seed(42)
n_days = 1500

returns = np.random.normal(0.0002, 0.015, n_days)
for i in range(1, len(returns)):
    returns[i] += 0.05 * returns[i-1]

prices = 100 * np.cumprod(1 + returns)
df = pd.DataFrame({'price': prices, 'returns': returns})

df['ret_5d'] = df['price'].pct_change(5)
df['vol_5d'] = df['returns'].rolling(5).std()
df['mom_5d'] = df['returns'].rolling(5).sum()
df['target'] = (df['returns'].shift(-1) > 0).astype(int)
df = df.dropna()

feature_cols = ['ret_5d', 'vol_5d', 'mom_5d']
X = df[feature_cols].values
y = df['target'].values

split = int(len(X) * 0.7)
scaler = StandardScaler()
X_train = scaler.fit_transform(X[:split])
X_test = scaler.transform(X[split:])
y_train, y_test = y[:split], y[split:]

model = GradientBoostingClassifier(n_estimators=50, max_depth=3, random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
probabilities = model.predict_proba(X_test)[:, 1]

print(f"Test accuracy: {(predictions == y_test).mean():.3f}")

## Part 1: Cost Model

In [ ]:
# ============================================================
# TRANSACTION COST MODEL
# ============================================================

class TransactionCostModel:
    """Model trading costs."""
    
    def __init__(self, commission_pct=0.001, spread_pct=0.0005, slippage_pct=0.0002):
        """
        Args:
            commission_pct: Commission per trade (0.1% = 0.001)
            spread_pct: Half bid-ask spread (0.05% = 0.0005)
            slippage_pct: Average slippage (0.02% = 0.0002)
        """
        self.commission = commission_pct
        self.spread = spread_pct
        self.slippage = slippage_pct
        
    def one_way_cost(self):
        """Cost for a single trade (buy or sell)."""
        return self.commission + self.spread + self.slippage
    
    def round_trip_cost(self):
        """Cost for buy + sell."""
        return 2 * self.one_way_cost()
    
    def calculate_costs(self, positions):
        """
        Calculate costs from position changes.
        
        Args:
            positions: Array of positions (-1, 0, or 1)
            
        Returns:
            Array of costs per period
        """
        position_changes = np.abs(np.diff(positions, prepend=0))
        costs = position_changes * self.one_way_cost()
        return costs
    
    def print_costs(self):
        print(f"Commission: {self.commission*100:.2f}%")
        print(f"Spread: {self.spread*100:.2f}%")
        print(f"Slippage: {self.slippage*100:.2f}%")
        print(f"One-way: {self.one_way_cost()*100:.2f}%")
        print(f"Round-trip: {self.round_trip_cost()*100:.2f}%")

# Test
cost_model = TransactionCostModel()
print("Default Cost Model:")
cost_model.print_costs()

## Part 2: Turnover Calculation

In [ ]:
# ============================================================
# TURNOVER CALCULATION
# ============================================================

print("\nTURNOVER ANALYSIS")
print("="*60)

def calculate_turnover(positions):
    """
    Calculate portfolio turnover.
    
    Turnover = sum of absolute position changes / 2
    (Divided by 2 because a flip from +1 to -1 is one round trip)
    """
    position_changes = np.abs(np.diff(positions, prepend=0))
    total_turnover = position_changes.sum() / 2
    daily_turnover = total_turnover / len(positions)
    annual_turnover = daily_turnover * 252
    
    return {
        'total': total_turnover,
        'daily': daily_turnover,
        'annual': annual_turnover
    }

# Binary positions from predictions
positions_binary = 2 * predictions - 1  # Convert 0/1 to -1/+1

turnover = calculate_turnover(positions_binary)
print(f"Total trades: {turnover['total']:.0f}")
print(f"Daily turnover: {turnover['daily']:.2%}")
print(f"Annual turnover: {turnover['annual']:.0f}x")

## Part 3: Strategy Performance with Costs

In [ ]:
# ============================================================
# BACKTEST WITH COSTS
# ============================================================

print("\nBACKTEST WITH TRANSACTION COSTS")
print("="*60)

test_returns = returns[split + 5:split + 5 + len(predictions)]  # Align with features
min_len = min(len(test_returns), len(predictions))
test_returns = test_returns[:min_len]
positions_binary = positions_binary[:min_len]

# Gross returns (before costs)
gross_returns = test_returns * positions_binary

# Net returns (after costs)
costs = cost_model.calculate_costs(positions_binary)
net_returns = gross_returns - costs

# Cumulative
cum_gross = np.cumprod(1 + gross_returns)
cum_net = np.cumprod(1 + net_returns)
cum_market = np.cumprod(1 + test_returns)

print(f"Gross Return: {(cum_gross[-1]-1)*100:.1f}%")
print(f"Net Return: {(cum_net[-1]-1)*100:.1f}%")
print(f"Market Return: {(cum_market[-1]-1)*100:.1f}%")
print(f"Total Costs: {costs.sum()*100:.2f}%")

In [ ]:
# ============================================================
# VISUALIZE COST IMPACT
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Cumulative returns
axes[0, 0].plot(cum_gross, label=f'Gross ({(cum_gross[-1]-1)*100:.0f}%)', linewidth=2)
axes[0, 0].plot(cum_net, label=f'Net ({(cum_net[-1]-1)*100:.0f}%)', linewidth=2)
axes[0, 0].plot(cum_market, label=f'Market ({(cum_market[-1]-1)*100:.0f}%)', alpha=0.7)
axes[0, 0].set_xlabel('Days')
axes[0, 0].set_ylabel('Cumulative Return')
axes[0, 0].set_title('Strategy Performance: Gross vs Net', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Cost drag over time
cumulative_costs = np.cumsum(costs)
axes[0, 1].fill_between(range(len(cumulative_costs)), cumulative_costs * 100, alpha=0.5, color='red')
axes[0, 1].set_xlabel('Days')
axes[0, 1].set_ylabel('Cumulative Costs (%)')
axes[0, 1].set_title('Cumulative Transaction Costs', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Position changes
position_changes = np.abs(np.diff(positions_binary, prepend=0))
axes[1, 0].bar(range(len(position_changes)), position_changes, width=1, alpha=0.5)
axes[1, 0].set_xlabel('Days')
axes[1, 0].set_ylabel('Position Change')
axes[1, 0].set_title('Trading Activity', fontweight='bold')

# Cost sensitivity
cost_levels = np.linspace(0.0001, 0.005, 20)
final_returns = []
for cost_pct in cost_levels:
    cm = TransactionCostModel(commission_pct=cost_pct/2, spread_pct=cost_pct/4, slippage_pct=cost_pct/4)
    costs_temp = cm.calculate_costs(positions_binary)
    net_temp = gross_returns - costs_temp
    final_returns.append((np.cumprod(1 + net_temp)[-1] - 1) * 100)

axes[1, 1].plot(cost_levels * 100, final_returns, 'o-', color='steelblue', linewidth=2)
axes[1, 1].axhline(0, color='gray', linestyle='--')
axes[1, 1].axhline((cum_market[-1]-1)*100, color='green', linestyle='--', label='Market')
axes[1, 1].set_xlabel('One-Way Cost (%)')
axes[1, 1].set_ylabel('Final Return (%)')
axes[1, 1].set_title('Cost Sensitivity Analysis', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 4: Reducing Turnover

In [ ]:
# ============================================================
# REDUCE TURNOVER WITH THRESHOLD
# ============================================================

print("\nREDUCING TURNOVER")
print("="*60)

def apply_threshold(probabilities, low=0.4, high=0.6):
    """
    Only trade when probability is outside threshold.
    Reduces turnover by requiring stronger signals.
    """
    positions = np.zeros(len(probabilities))
    current_pos = 0
    
    for i, prob in enumerate(probabilities):
        if prob > high:
            current_pos = 1
        elif prob < low:
            current_pos = -1
        # else: keep current position
        positions[i] = current_pos
    
    return positions

# Compare turnover
positions_threshold = apply_threshold(probabilities[:min_len], low=0.4, high=0.6)

turnover_orig = calculate_turnover(positions_binary)
turnover_threshold = calculate_turnover(positions_threshold)

print(f"Original turnover: {turnover_orig['annual']:.0f}x annual")
print(f"With threshold: {turnover_threshold['annual']:.0f}x annual")
print(f"Reduction: {(1 - turnover_threshold['annual']/turnover_orig['annual'])*100:.0f}%")

# Performance comparison
gross_threshold = test_returns * positions_threshold
costs_threshold = cost_model.calculate_costs(positions_threshold)
net_threshold = gross_threshold - costs_threshold

print(f"\nNet return (original): {(np.cumprod(1+net_returns)[-1]-1)*100:.1f}%")
print(f"Net return (threshold): {(np.cumprod(1+net_threshold)[-1]-1)*100:.1f}%")

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║              DAY 2 COMPLETE: TRANSACTION COSTS                   ║
╠══════════════════════════════════════════════════════════════════╣
║  ✓ Transaction cost model (commission, spread, slippage)        ║
║  ✓ Turnover calculation                                         ║
║  ✓ Cost sensitivity analysis                                    ║
║  ✓ Turnover reduction techniques                                ║
╚══════════════════════════════════════════════════════════════════╝

Tomorrow: Day 3 - Position Sizing
""")